# Daily Very-Useful Channel


## Введение
Этот проект автоматизирует процесс получения финансовых данных от Центрального банка России (ЦБ РФ) и Московской биржи (Мосбиржа), а затем отправляет эту информацию в указанный Telegram-канал. Скрипт выполняется ежедневно по будням в установленное время, предоставляя актуальные данные о валютных курсах и акциях. Используя принцип работы, изложенный в данном проекте, можно отправлять в Telegram-канал любую информацию, используя возможности, которые предоставляет GitHub Actions. 

## Цели
- Получать ежедневные курсы валют от ЦБ РФ.
- Получать ежедневную информацию о фондовом рынке от Мосбиржи.
- Отправлять полученную информацию в Telegram-канал.
"""

## 1. Создание Телеграм бота и Телеграм канала, подключение к API 

Перед тем, как переходить к автоматизации, создайте Телеграм-бот и Телеграм-канал. Без этих инструментов проект не заработает. 
 - Документация Сервиса BotFather для создания бота - https://core.telegram.org/bots
 - Пример инструкции для создания телеграмканала - https://smmplanner.com/blog/kak-sozdat-kanal-v-tielieghramie-polnaia-instruktsiia/
 - Ссылка на инструкцию ИСС Мосбиржи - https://fs.moex.com/files/8888
 - Ссылка на документацию ЦБРФ - https://cbr.ru/development/DWS/
 - ДЛЯ УСПЕШНОЙ РЕАЛИЗАЦИИ ПРОЕКТА НЕОБХОДИМО ИМЕТЬ АККАУНТ НА GITHUB. 
  
  Ссылка на инструкцию работы с системами контроля версий: https://www.youtube.com/watch?v=HL1QmInke7M&t=286s
  
## 2. Устанавливаем библиотеки
Этот проект требует следующих Python пакетов:
- `requests`
- `pandas`
- `json`
- `os`
- `datetime`
- `numpy`
- `itertools`
- `pprint`
- `python-dotenv`
- `tabulate`

####  MacOs и Linux:

source venv/bin/activate
pip install -r requirements.txt

#### Windows:

venv\Scripts\activate
pip install -r requirements.txt

Убедитесь, что у вас есть файл `.env` со следующими переменными:
```
TELEGRAM_TOKEN=<ваш-telegram-bot-token>
TELEGRAM_CENNAL_ID=<ваш-telegram-channel-id>
```

In [ ]:
import requests
import pandas as pd
import json 
import os
from datetime import datetime 
import numpy as np
from itertools import count
from  pprint import pprint
from dotenv import load_dotenv
import tabulate

load_dotenv() # функция используется для загрузки данных окружения 


## 3. Функции сбора данных

### Получение курсов валют от ЦБ РФ
Функция `get_cur` получает ежедневные курсы валют от Центрального банка России (ЦБ РФ) и форматирует данные в удобную для чтения таблицу.

### Получение информации о фондовом рынке от Мосбиржи
Функция `get_market` получает данные о фондовом рынке от Московской биржи (Мосбиржа), проходя через пагинированные результаты для сбора всех доступных данных.
"""

In [ ]:
def get_cur():
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'
    response = requests.get(url) # делаем запрос к сайту ЦБРФ
    currencies = response.json() # преобразуем в json-стандарт
    fr_currencies = (
        currencies["Valute"]["KZT"], currencies["Valute"]["CNY"], 
        currencies["Valute"]["AED"], currencies["Valute"]["USD"], currencies["Valute"]["EUR"]
    )
# cоздаем переменную с нужными валютами (в нашем случае 3 "дружественных" и 2 "мировые" валюты)
    df_currenc = pd.DataFrame(fr_currencies) # создаем датафрэйм
    df_cur = (
        df_currenc[['CharCode', 'Nominal', 'Value']]
        .rename(columns={'CharCode':'Валюта', 'Nominal':'Кол-во', 'Value':'За ед.'})
    ) #вытаскиваем и переименовываем нужные столбцы и переименовываем
    
    # Форматируем сообщение в Markdown
    markdown_message = "*Курсы валют от ЦБРФ:*\n"
    for index, row in df_cur.iterrows():
        markdown_message += f"- *{row['Валюта']}*: {row['Кол-во']} = {row['За ед.']} руб.\n"
    
    return markdown_message

# Функция для получения всех данных с API MOEX
def get_market():
    url_hist = "http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json"
    all_data = []  # Список для хранения всех данных

    for stz_row in count(start=0, step=100):
        # Формирование URL с параметром start
        url = f"{url_hist}?start={stz_row}"
        # Выполнение GET-запроса к API
        response = requests.get(url)
        # Разбор JSON-ответа
        result_h = json.loads(response.text)
        # Извлечение данных
        dt_r = result_h['history']['data']
        if not dt_r:
            break  # Выход из цикла, если данные закончились
        # Добавление данных в общий список
        all_data.extend(dt_r)

    # Извлечение имен столбцов
    dt_c = result_h['history']['columns']
    # Создание DataFrame из всех данных
    df_sh = pd.DataFrame(all_data, columns=dt_c)
    # Возвращаем DataFrame
    return df_sh

## 4. Функции обработки данных

### Фильтрация акций голубых фишек
Функция `filter_blue_chips` фильтрует данные о фондовом рынке, оставляя только акции голубых фишек, которые представляют собой хорошо зарекомендовавшие себя и финансово устойчивые компании.
"""

In [ ]:
# Функция для фильтрации данных голубых фишек (данные обновляются на бирже регулярно, поэтому изменения нужновносить вручную)
def filter_blue_chips(df_sh):
    df_market_bc = df_sh[['SHORTNAME', 'LEGALCLOSEPRICE', 'TRENDCLSPR']]
    df_market_bc = (
        df_market_bc[(df_market_bc['SHORTNAME'] == 'АЛРОСА ао')| (df_market_bc['SHORTNAME'] == 'ГАЗПРОМ ао') | 
        (df_market_bc['SHORTNAME']=='Сбербанк') | (df_market_bc['SHORTNAME'] == 'Татнфт 3ао') | 
        (df_market_bc['SHORTNAME'] == 'ГМКНорНик')|(df_market_bc['SHORTNAME'] =='ММК')|(df_market_bc['SHORTNAME'] =='Магнит ао')|
        (df_market_bc['SHORTNAME'] == 'Полюс')|(df_market_bc['SHORTNAME'] =='НЛМК ао')|(df_market_bc['SHORTNAME'] == 'Новатэк ао')|
        (df_market_bc['SHORTNAME'] == 'Роснефть')|(df_market_bc['SHORTNAME'] == 'СевСт-ао')|(df_market_bc['SHORTNAME'] == 'ТКСХолд ао')|
        (df_market_bc['SHORTNAME'] =='Сургнфгз')|(df_market_bc['SHORTNAME'] == 'ЛУКОЙЛ')
    ]).rename(columns={'SHORTNAME':'Назв.', 'LEGALCLOSEPRICE':'Цена', 'TRENDCLSPR':'Изм.(%)'})
    return df_market_bc

## 5. Интеграция с Telegram ботом

### Отправка сообщений в Telegram
Функция `send_message` отправляет форматированное сообщение в указанный Telegram-канал с использованием API Telegram Bot. Запуск основной фенкции осуществляется при помощи кострукции "if __name__ == '__main__':"
"""

In [ ]:
# Функция для отправки сообщения в Telegram

def send_message(message):
    
    TELEGRAM_TOKEN = os.getenv('TELEGRAM_TOKEN')
    TELEGRAM_CENNAL_ID = os.getenv('TELEGRAM_CENNAL_ID')
    
    
    url = f'https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage'
    
    params = {
        'chat_id': TELEGRAM_CENNAL_ID,
        'text': message
    }
    
    res = requests.post(url, params=params)
    return res.json()

if __name__ == '__main__':
    
    currencies = get_cur() # Получение данных с ЦБРФ
    df_sh = get_market()  # Получение данных с Мосбиржи
    
    # Фильтрация данных голубых фишек
    df_market_bc = filter_blue_chips(df_sh)
    
    # Формируем сообщения для отправки
    currency_message = "Курсы валют от ЦБРФ:\n\n"
    currency_message += currencies
    
    stock_message = "*Акции голубых фишек Мосбиржи:*\n\n"
    for index, row in df_market_bc.iterrows():
        stock_message += f"*{row['Назв.']}*\n"
        stock_message += f"Цена: {row['Цена']}\n"
        stock_message += f"Изм.(%): {row['Изм.(%)']}\n\n"
    
    # Отправляем сообщения в Telegram
    send_message(currency_message)
    send_message(stock_message)

## 6. Создаем Файл автоматизации в Github Actions

### Цели
- Автоматически выполнять скрипт ежедневно по будням.
- Обеспечить надежную доставку данных в Telegram-канал.

Документация Github Actions: https://docs.github.com/en/actions

Документация об использовании Github Secrets: https://docs.github.com/en/actions/security-guides/using-secrets-in-github-actions

Для автоматизации создайте файл `.github/workflows/run_daily.yml` в папке проекта на своем компьютере. Кроме того, для того, чтобы скрипт работал надежнее, создайте отдельный файл с функциями автоматизации с расширением .py (в моем случае - это app.py).

Использование GitHub Actions для автоматизации позволяет надежно и эффективно выполнять задачи по сбору и отправке данных. Настройка ежедневных задач помогает гарантировать, что ваш Telegram-канал всегда будет иметь актуальную информацию о курсах валют и фондовом рынке.

NB: Если больше нет нужды держать скрипт работающим ,его можно перевести на ручное управление, заменив в run_daily.yml значение on:  с schedule:
    - cron:...
на workflow_dispatch


In [ ]:
# Файл создавайте с расширением .yml

name: Daily Very-Useful Channel
on:
  schedule:
    - cron: '5 16 * * 1-5'

env:
  TELEGRAM_TOKEN: ${{ secrets.TELEGRAM_TOKEN }}
  TELEGRAM_CENNAL_ID: ${{ secrets.TELEGRAM_CENNAL_ID}}

jobs:
    Daily-Run:
        runs-on: ubuntu-latest
        steps:
            - name: Checkout Code
              uses: actions/checkout@v4
            - name: Install Dependencies
              run: pip install -r requirements.txt
            - name: Run App
              run: python app.py 


#### Описание шагов

- name: Checkout Code: Этот шаг использует действие actions/checkout@v4 для клонирования репозитория на виртуальную машину.
- name: Install Dependencies: Этот шаг устанавливает все необходимые зависимости, перечисленные в файле requirements.txt.
- name: Run App: Этот шаг запускает основной скрипт app.py, который собирает данные и отправляет их в Telegram-канал.
- Переменные окружения
- TELEGRAM_TOKEN: Токен для вашего Telegram бота.
- TELEGRAM_CENNAL_ID: ID вашего Telegram канала.
- Эти переменные задаются с использованием GitHub Secrets для обеспечения безопасности.
  

